In [1]:
import numpy as np
import pandas as pd

# Define IAM role
import boto3
import re
import sagemaker
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
bucket_name = 'swati-ml-sagemaker'
training_file_key = 'iris/iris_train.csv'
validation_file_key = 'iris/iris_validation.csv'

s3_model_output_location = r's3://{0}/iris/model'.format(bucket_name)
s3_training_file_location = r's3://{0}/{1}'.format(bucket_name,training_file_key)
s3_validation_file_location = r's3://{0}/{1}'.format(bucket_name,validation_file_key)

In [3]:
print(s3_model_output_location)
print(s3_training_file_location)
print(s3_validation_file_location)

s3://swati-ml-sagemaker/iris/model
s3://swati-ml-sagemaker/iris/iris_train.csv
s3://swati-ml-sagemaker/iris/iris_validation.csv


In [4]:
def write_to_s3(filename, bucket, key):
    with open(filename,'rb') as f: # Read in binary mode
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [5]:
write_to_s3('iris_train.csv',bucket_name,training_file_key)
write_to_s3('iris_validation.csv',bucket_name,validation_file_key)

In [6]:
use_spot_instances = True
max_run = 3600 # in seconds
max_wait = 7200 if use_spot_instances else None # in seconds

job_name = 'xgboost-iris-v1'

checkpoint_s3_uri = None

if use_spot_instances:
    checkpoint_s3_uri = f's3://{bucket_name}/iris/checkpoints/{job_name}'
    
print (f'Checkpoint uri: {checkpoint_s3_uri}') 

Checkpoint uri: s3://swati-ml-sagemaker/iris/checkpoints/xgboost-iris-v1


In [7]:
sess = sagemaker.Session()
role = get_execution_role()
print(role)

arn:aws:iam::637423580352:role/service-role/AmazonSageMaker-ExecutionRole-20240325T165146


In [8]:
container = sagemaker.image_uris.retrieve("xgboost",sess.boto_region_name,version="1.2-2")

print (f'Using XGBoost Container {container}')

Using XGBoost Container 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-2


### Build Model

In [9]:
estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=s3_model_output_location,
    sagemaker_session=sess,
    base_job_name = job_name,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
    checkpoint_s3_uri=checkpoint_s3_uri)

In [10]:
estimator.set_hyperparameters(max_depth=5,
                              objective="multi:softmax",
                              eval_metric="mlogloss",
                              num_class=3,
                              num_round=100,
                              early_stopping_rounds=10)

In [11]:
estimator.hyperparameters()

{'max_depth': 5,
 'objective': 'multi:softmax',
 'eval_metric': 'mlogloss',
 'num_class': 3,
 'num_round': 100,
 'early_stopping_rounds': 10}

In [12]:
# content type can be libsvm or csv for XGBoost
training_input_config = sagemaker.session.TrainingInput(
    s3_data=s3_training_file_location,
    content_type='csv',
    s3_data_type='S3Prefix')

validation_input_config = sagemaker.session.TrainingInput(
    s3_data=s3_validation_file_location,
    content_type='csv',
    s3_data_type='S3Prefix'
)

data_channels = {'train': training_input_config, 'validation': validation_input_config}

In [13]:
print(training_input_config.config)
print(validation_input_config.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://swati-ml-sagemaker/iris/iris_train.csv', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'csv'}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://swati-ml-sagemaker/iris/iris_validation.csv', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'csv'}


### Train model

In [14]:
estimator.fit(data_channels)

INFO:sagemaker:Creating training-job with name: xgboost-iris-v1-2024-05-19-17-19-00-521


2024-05-19 17:19:00 Starting - Starting the training job...
2024-05-19 17:19:16 Starting - Preparing the instances for training...
2024-05-19 17:19:43 Downloading - Downloading input data...
2024-05-19 17:20:04 Downloading - Downloading the training image...
2024-05-19 17:20:44 Training - Training image download completed. Training in progress..[2024-05-19 17:20:54.395 ip-10-0-167-228.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-05-19:17:20:54:INFO] Imported framework sagemaker_xgboost_container.training
[2024-05-19:17:20:54:INFO] Failed to parse hyperparameter eval_metric value mlogloss to Json.
Returning the value itself
[2024-05-19:17:20:54:INFO] Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
[2024-05-19:17:20:54:INFO] No GPUs detected (normal if no gpus installed)
[2024-05-19:17:20:54:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-05-19:17:20:54:INFO] Determined delimiter of CSV input is ','
[2

### Deploy Model

In [15]:
predictor = estimator.deploy(initial_instance_count=1,
                             instance_type='ml.m5.xlarge',
                             endpoint_name = job_name)

INFO:sagemaker:Creating model with name: xgboost-iris-v1-2024-05-19-17-22-13-111
INFO:sagemaker:Creating endpoint-config with name xgboost-iris-v1
INFO:sagemaker:Creating endpoint with name xgboost-iris-v1


------!

### Run predictions

In [16]:
from sagemaker.serializers import CSVSerializer
predictor.serializer = CSVSerializer()
predictor.predict([[4.8,3.4,1.6,0.2],[4.8,3.4,1.6,0.2],[5.8,2.7,4.1,1.0]])

b'0.0\n0.0\n1.0\n'